In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab04.ipynb")

<a id='verytop'></a>

# Lab 4: Applications of Eigenvectors and Eigenvalues

## Due Date: Thurs, Oct 16th 11:59 PM on Gradescope


### Detailed Submission Instructions Are Provided at the end of this Notebook


## Collaboration Policy

A key step in learning and retention is **creating solutions on your own.**   Below are examples of acceptable vs unacceptable use of resources and collaboration when doing lab assignments in CSCI 2820.


The following would be some **examples of cheating** when working on HW assignments in CSCI 2820.  Any of these constitute a **violation of the course's collaboration policy and will result in an F in the course and a trip to the honor council**.   


 - Consulting web pages that may have a solution to a given lab problem or one similar is cheating.  However, consulting notes from the class videos, and web pages that explain the material taught in class but do NOT show a solution to the lab problem in question are permissible to view.  Clearly, there's a fuzzy line here between a valid use of resources and cheating. To avoid this line, one should merely consult the course videos, the course textbooks, and references that contain syntax and/or formulas.
 - Copying a segment of code or math solution of three lines or more from another student from a printout, handwritten copy, or by looking at their computer screen 
 - Allowing another student to copy a segment of your code or math solution of three lines or more
 - Taking a copy of another student's work (or a solution found online) and then editing that copy
 - Reading someone else’s solution to a problem on the lab before writing your own.
 - Asking someone to write all or part of a program or solution for you.
 - Asking someone else for the code necessary to fix the error for you, other than for simple syntactical errors
 


On the other hand, the following are some **examples of things which would NOT usually be
considered to be cheating**:
 - Working on a lab problem on your own first and then discussing with a classmate a particular part in the problem solution where you are stuck.  After clarifying any questions you should then continue to write your solution independently.
 - Asking someone (or searching online) how a particular construct in the language works.
 - Asking someone (or searching online) how to formulate a particular construct in the language.
 - Asking someone for help in finding an error in your program.  
 - Asking someone why a particular construct does not work as you expected in a given program.
   

To test whether you are truly doing your own work and retaining what you've learned you should be able to easily reproduce from scratch and explain a lab solution that was your own when asked in office hours by a TA/Instructor or on a quiz/exam.   


If you have difficulty in formulating the general solution to a problem on your own, or
you have difficulty in translating that general solution into a program, it is advisable to see
your instructor or teaching assistant rather than another student as this situation can easily
lead to a, possibly inadvertent, cheating situation.

We are here to help!  Visit office Hours and/or post questions on Piazza!



## Grading
Grading is broken down into autograded answers and manually graded answers. 

For autograded answers, the results of your code are compared to provided and/or hidden tests.

For manually graded answers you must show and explain all steps.  Graders will evaluate how well you answered the question and/or fulfilled the requirements of the question.


<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


In [ ]:
# import useful libraries
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp

sp.init_printing(use_unicode=True)


# Function needed to run in-notebook tests
import hashlib

def get_hash(num):
    """Helper function for assessing correctness"""
    return hashlib.md5(str(num).encode()).hexdigest()


def get_array_hash_normalized(arr):
    """
    Hash numpy array ignoring dtype differences (values only),
    treating -0.0 as 0.0 and canonicalizing NaNs.
    """
    arr = np.ascontiguousarray(arr, dtype=np.float64)

    # View as unsigned integers so we can manipulate bits
    view = arr.view(np.uint64)

    # Mask for the exponent+mantissa (everything except the sign bit)
    mant_exp_mask = (1 << 63) - 1

    # For entries that are exactly zero (pos or neg), clear sign bit
    zero_mask = (view & mant_exp_mask) == 0
    view[zero_mask] = 0

    # Canonicalize NaNs: all NaNs get the same payload
    nan_mask = np.isnan(arr)
    if nan_mask.any():
        arr[nan_mask] = np.float64(np.nan)

    return hashlib.md5(arr.tobytes() + str(arr.shape).encode()).hexdigest()

## Recall: Matrix and Vector Operations in SymPy

For this lab we will return to using Symbolic Python.  This will help ensure we don't have floating point errors when dealing with different matrices.


Recall: to create the matrix
\begin{equation}
\mathbf{B} = \left[\begin{array}{r} 1 &  3 \\  2 & 4 \\ -3 &5 \end{array}\right]
\end{equation}
in SymPy we can use the following syntax:

In [ ]:
B = sp.Matrix([[1,3],[2,4], [-3,5]])
B

Recall: We can find RREF using:


In [ ]:
B.rref()

For an invertible matrix, you can find the inverse in Sympy using the following syntax:  `**-1`


In [ ]:
A = sp.Matrix([[1,3],[2,4]])
A

In [ ]:
A**-1

# Eigenvectors and Eigenvalues

We first consider a matrix that represents a shear along the line $x_1=-x_2$.

$$
\begin{equation}
A = \left[ \begin{array}{rr} 2 & -1  \\ -1 & 2 \end{array}\right]
\end{equation}
$$



In [ ]:
# Here we will use Symbolic Python to define the matrix A:

A = sp.Matrix([[2,-1],[-1,2]])
A

Let's visualize the effect of this matrix transformation on a given vector:




Let $$
\begin{equation}
x = \left[ \begin{array}{r} 3  \\ 1 \end{array}\right]
\end{equation}.
$$

In [ ]:
x = sp.Matrix([[3],[1]])
x

In [ ]:
A@x

The following code plots the 2D vector $\mathbf{x}$ and its image $A\mathbf{x}$ under the linear transformation defined by A.

In [ ]:

def plot_transformation(A, x):
    """
    Plot the 2D vector x and its image Ax under the linear transformation defined by A.

    Parameters:
        A (sp.Matrix): a 2x2 sympy matrix
        x (sp.Matrix): a 2x1 sympy column vector
    """
    # Compute Ax
    Ax = A * x

    # Convert to numerical values for plotting
    x_vals = np.array(x, dtype=float).flatten()
    Ax_vals = np.array(Ax, dtype=float).flatten()

    # Set up plot
    fig, ax = plt.subplots()
    t = np.linspace(-6, 6, 100)
    options = {"head_width": 0.1, "head_length": 0.2, "length_includes_head": True}

    # Plot x (blue) and Ax (red)
    ax.arrow(0, 0, x_vals[0], x_vals[1], fc='b', ec='b', **options)
    ax.arrow(0, 0, Ax_vals[0], Ax_vals[1], fc='r', ec='r', **options)

    # Plot dashed guide line (example: x1 = -x2)
    ax.plot(t, -t, ls=':')

    # Set limits and formatting
    ax.set_xlim(-3, 6)
    ax.set_ylim(-3, 4)
    ax.set_aspect('equal')
    ax.set_xticks(np.arange(-1, 7, step=1))
    ax.set_yticks(np.arange(-3, 5, step=1))

    # Label vectors
    ax.text(x_vals[0] / 2, x_vals[1] / 2 + 0.2, '$X$', color='b')
    ax.text(Ax_vals[0] / 2, Ax_vals[1] / 2 - 0.2, '$AX$', color='r')
    ax.text(0.8, -2.5, '$x_1=-x_2$')

    # Axes and grid
    ax.axvline(color='k', linewidth=1)
    ax.axhline(color='k', linewidth=1)
    ax.grid(True, ls=':')

    plt.show()


In [ ]:
plot_transformation(A, sp.Matrix([[3],[1]]))

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## <span style='color:Red'>   Question 1a (2 pts) ###


Enter different inputs for $x$ into the `plot_transformation` function to **visually** find two linearly independent eigenvectors, $\mathbf{v_1}$ and $\mathbf{v_2}$ of $A$.   


i).  Which of the following is an eigenvector of $A$?  Save the correct answer for $\mathbf{v_1}$ below as a Sympy 2x1 matrix

$\mathbf{v_1} = \left[ \begin{array}{r} 3 \\ 1 \end{array}\right]$  $\hspace{3mm}$ or $\hspace{5mm}$ $\mathbf{v_1} = \left[ \begin{array}{r} 2 \\ 1 \end{array}\right] \hspace{3mm}$ or $\hspace{5mm}$ $\mathbf{v_1} = \left[ \begin{array}{r} 1 \\ -1 \end{array}\right] \hspace{3mm} or \hspace{3mm}  \mathbf{v_1}  \left[ \begin{array}{r} -1 \\ 2 \end{array}\right]$

ii).  What is the corresponding eigenvalue for $\mathbf{v_1}$?

iii).      Which of the following is another eigenvector of $A$ that is linearly independent from the first?  Save the correct answer for $\mathbf{v_1}$ below as a Sympy 2x1 matrix

$\mathbf{v_2} = \left[ \begin{array}{r} 1 \\ -3 \end{array}\right]$  $\hspace{3mm}$ or $\hspace{5mm}$ $\mathbf{v_2} = \left[ \begin{array}{r} 1 \\ 2 \end{array}\right] \hspace{3mm}$ or $\hspace{5mm}$ $\mathbf{v_2} = \left[ \begin{array}{r} 1 \\ 5 \end{array}\right] \hspace{3mm} or \hspace{3mm}  \mathbf{v_2} \left[ \begin{array}{r} 1 \\ 1 \end{array}\right]$                                
iv).  What is the corresponding eigenvalue for $\mathbf{v_2}$?



In [ ]:

#Enter your answers for eigenvectors as a Sympy 2x1 matrix i.e. v1 = sp.Matrix([[a],[b]])

v1 = ...

lambda_1 = 3

v2 = sp.Matrix([[1],[1]])

lambda_2 = 1


In [ ]:
grader.check("q1a")

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## <span style='color:Red'>   Question 1b (2 pts) ###


Since $\mathbf{v_1}$ and $\mathbf{v_2}$ are linearly independent and span $\mathbb{R}^2$ they form a basis.

Given the vector $\mathbf{x} = \left[ \begin{array}{r} 3 \\ 1 \end{array}\right]$,

rewrite as a linear combination of $\mathbf{v_1}$ and $\mathbf{v_2}$.

i).  That is, find $c_1, c_2$ such that $c_1\mathbf{v_1} + c_2\mathbf{v_2}$.

ii).  We can also think of this as changing basis.  Let $\mathcal{B}  = \{\mathbf{v_1}, \mathbf{v_2}\}$.  What is $\{\mathbf{x}\}_{\mathcal{B}}$?  (Give your answer as a Sympy 2x1 Matrix)




In [ ]:


...


c1 = ...

c2 = ...

# Give your answer for x_B as a SymPy 2x1 Matrix
x_B = ...

x_B




In [ ]:
grader.check("q1b")

Let's look at a visual representation of the linear combination $X = V_1 + 2V_2$.

We can now understand, and compute, the product $AX$ by evaluating the products of $A$ with its eigenvectors. 

$$
\begin{equation}
AX = A(V_1 + 2V_2) = AV_1 + 2AV_2 = \lambda_1V_1 + 2\lambda_2V_2 = 3V_1 + 2V_2
\end{equation}
$$

Since the action of an $n\times n$ matrix $A$ on its eigenvectors is easy to understand, we might try to understand the action of a matrix on an arbitrary vector $X$ by writing $X$ as a linear combination of eigenvectors. This will always be possible *if the eigenvectors form a basis for* $\mathbb{R}^n$.  Suppose $\{V_1, V_2, ..., V_n\}$ are the eigenvectors of $A$ and they do form a basis for $\mathbb{R}^n$.  Then for any vector $X$, we can write 
$X = c_1V_1 + c_2V_2 + ... c_nV_n$.  The product $AX$ can easily be computed then by multiplying each eigenvector component by the corresponding eigenvalue.

$$
\begin{eqnarray*}
AX & = & A(c_1V_1 + c_2V_2 + ... c_nV_n) \\
   & = & c_1AV_1 + c_2AV_2 + ... c_nAV_n \\
   & = & c_1\lambda_1V_1 + c_2\lambda_2V_2 + ... c_n\lambda_nV_n
\end{eqnarray*}
$$

## Discrete Dynamical Systems

### Calculating Eigenvalues and Eigenvectors in Sympy

To find the eigenvectors of a matrix, in Sympy we can use `eigenvects()`. This returns a list of tuples of the form (eigenvalue, algebraic_multiplicity, [eigenvectors]).

In [ ]:
A = sp.Matrix([[1,2],[4,5]])

In [ ]:
eigen_info= A.eigenvects()
eigen_info

In [ ]:
# To list just the eigenvectors:

eigenvectors = []
for eigenvalue, multiplicity, vecs in eigen_info:
    eigenvectors.extend(vecs)

eigenvectors

We may not recognize the eigenvectors as those we found previously, but recall that eigenvectors are not unique.  The $\texttt{eig}$ function scales all the eigenvectors to unit length, and we arrive at the same result if we scale our choice of eigenvector.

# Diagonal factorization

The calculation that we used to verify the eigenvalues and eigenvectors is also very useful to construct another important matrix factorization.  Suppose that $A$ is an $n\times n$ matrix, $D$ is a diagonal matrix with the eigenvalues of $A$ along its diagonal, and $S$ is the $n\times n$ matrix with the eigenvectors of $A$ as its columns.  We have just seen that $AS=SD$.  If $S$ is invertible, we may also write $A=SDS^{-1}$, which is known as the **diagonalization** of $A$.

The diagonalization of $A$ is important because it provides us with a complete description of the action of $A$ in terms of its eigenvectors.  Consider an arbitrary vector $X$, and the product $AX$, computed by using the three factors in the diagonalization.

- $S^{-1}X$ computes the coordinates of $X$ in terms of the eigenvectors of $A$.
- Multiplication by $D$ then simply scales each coordinate by the corresponding eigenvalue.
- Multiplication by $S$ gives the results with respect to the standard basis.

This understanding does not provide a more efficient way of computing the product $AX$, but it does provide a much more general way of understanding the result of a matrix-vector multiplication.  As we will see in the next section, the diagonalization also provides a significant shortcut in computing powers of $A$.  

It should be noted that diagonalization of an $n\times n$ matrix $A$ is not possible when the eigenvectors of $A$ form a linearly dependent set.  In that case the eigenvectors do not span $\mathbb{R}^n$, which means that not every $X$ in $\mathbb{R}^n$ can be written as a linear combination of the eigenvectors.  In terms of the computation above, $S$ will not be invertible exactly in this case.

**Question:** Find the diagonalization of the matrix from **Question 1**.

$$
\begin{equation}
A = \left[ \begin{array}{rr} 2 & -1  \\ -1 & 2 \end{array}\right]
\end{equation}
$$

**Exercise 2:** Use the diagonailization to write A^(100) as a product of 3 matrices.


In [ ]:
## Code solution here.

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


### Submission Instructions

Before proceeding any further, **save this notebook.**

Then run the cell below to double check that you don't have any spaces between dollar signs and text when writing LaTeX:

In [ ]:
# Run this cell before you run the 'grader.export()' cell below.  
# It will search for LaTeX errors that will cause the LaTeX compiler to fail.  

import simple_latex_checker as slc

nb = slc.Nb_checker()
nb.run_check("lab04.ipynb")

After running the `grader.export()` cell provided below, **2 files will be created**: a zip file and pdf file.  You can download them using the links provided below OR by finding them in the same folder where this juptyer notebook resides in your JuptyerHub.

To receive credit on this assignment, **you must submit BOTH of these files
to their respective Gradescope portals:** 


* **Lab 4 Autograded**: Submit the zip file that is output by the `grader.export()` cell below to the Lab 4 Autograded assignment in Gradescope.

* **Lab 4 Manually Graded**: Submit your lab04.PDF to the Lab 4 Manually Graded assignment in Gradescope.  **It is your responsibility to fully review your PDF file before submitting and make sure that all your lines of code are visible and any LaTeX has correctly compiled and is fully viewable.**  **YOU MUST SELECT THE PAGES CORRESPONDING TO EACH QUESTION WHEN YOU UPLOAD TO GRADESCOPE.** If not, you will lose points.    

[TROUBLESHOOTING TIPS](https://docs.google.com/document/d/1ndr3Wj1PSF5qzlLMaBJznwh6QGeEXjd5TAJ6nf9EJvo/edit?usp=sharing)  If you are having any issues compiling your assignment, please read through these troubleshooting tips first, then post any questions on Piazza.  

**You are responsible for ensuring your submission follows our requirements. We will not be granting regrade requests nor extensions to submissions that don't follow instructions.** If you encounter any difficulties with submission, please don't hesitate to reach out to staff prior to the deadline.

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

AFTER running the cell below, click on <a href='lab04.pdf' download>this link to download the PDF </a> to upload to Gradescope.  There will be a separate link that appears after running the cell below with a link to download the zip file to upload to Gradescope.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)